In [61]:
from dotenv import load_dotenv
import urllib.parse as urlparse
import requests
import json
import os

class Untappd:

    def __init__(self, clientId = "", clientSecret = ""):
        load_dotenv(dotenv_path="../.env")
        self._clientId = os.getenv("CLIENT_ID")
        self._clientSecret = os.getenv("CLIENT_SECRET")
        self._baseUrl = "https://api.untappd.com/v4/"


    def keys(self):
        return {
            "client_id": self._clientId,
            "client_secret": self._clientSecret
        }

    def findBrewery(self, brewery="", writeFile=False):
        methodUrl = self._baseUrl + "search/brewery"
        params = {
            "q": brewery
        }
        requestUrl = methodUrl + "?" + urlparse.urlencode({**params, **self.keys()})
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        breweries = [] 
        
        if response.status_code == 200:
            content = response.json()
            
            
            if content["response"]["found"] > 0:
                breweries = content["response"]["brewery"]["items"]
        
        return breweries, counter, response
    
    def breweryInfo(self, breweryId="", writeFile=False):
        methodUrl = self._baseUrl + "brewery/info/"
        requestUrl = methodUrl + str(breweryId) + "?" + urlparse.urlencode(self.keys())
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        
        if response.status_code == 200:
            content = response.json()
            
            brewery = content["response"]
        
        return brewery, counter, response




### Create a brewery dataset

In [55]:
import numpy as np
import pandas as pd
import pickle

try:
    with open ("../data/all_breweries.pkl", "rb") as fp:
        all_breweries = pickle.load(fp)
except:
    print( "No file existed, creating one!" )
    breweries = pd.read_csv("../data/Texas Breweries.csv")
    brewpubs = pd.read_csv("../data/Texas Brewpubs.csv")
    brewery_list = list(breweries.T.to_dict().values())
    all_breweries = brewery_list + list( brewpubs.T.to_dict().values() )

    with open('../data/all_breweries.pkl', 'wb') as fp:
        pickle.dump(all_breweries, fp)
        
display( all_breweries[0:2] )

[{'Name': '111 Brewing',
  'Street Address': 111,
  'Road': 'S FANNIN AVENUE',
  'City': 'DENISON',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '4/10/2019',
  'Status': 'In the works'},
 {'Name': '2 Docs Brewing Co',
  'Street Address': 502,
  'Road': 'TEXAS AVENUE',
  'City': 'LUBBOCK',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '10/5/2018',
  'Status': 'Open'}]

Due to untappd's 100 requests an hour, i'll need to run this over a couple of hours.  Thus saving results for future reference.

In [62]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

untappd = Untappd()
 
for brewery in all_breweries:
    if "untappd" not in brewery:
        brewery_result, counter, response = untappd.findBrewery(brewery["Name"])
        
        if len( brewery_result ) > 0:
            brewery["untappd"] = brewery_result
        else:
            brewery["untappd"] = [{"no_results":True}]

        clear_output(wait=True)
        display(counter)
        if counter == 0:
            break
        
#         break

#Save results
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

print( "sample output" )
print( all_breweries[0] )

0

sample output
{'Name': '111 Brewing', 'Street Address': 111, 'Road': 'S FANNIN AVENUE', 'City': 'DENISON', 'State': 'TX', 'Brewery License': 'Brewery', 'License Date': '4/10/2019', 'Status': 'In the works', 'untappd': [{'brewery': {'brewery_id': 66888, 'beer_count': 65, 'brewery_name': 'Milepost 111 Brewing Company', 'brewery_slug': 'milepost-111-brewing-company', 'brewery_page_url': '/milepostbeer', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Cashmere', 'brewery_state': 'WA', 'lat': 47.5244, 'lng': -120.47}}}]}


In [52]:
import pickle
import pandas as pd

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)
 
# Normalized the results
brewery_parsed = []
for brewery in all_breweries:
    if "untappd" in brewery:
        for entries in brewery["untappd"]:
            brewery_parsed.append({
                "Name":brewery["Name"],
                "Street Address":brewery["Street Address"],
                "Road":brewery["Road"],
                "City":brewery["City"],
                "State":brewery["State"],
                "Brewery License":brewery["Brewery License"],
                "License Date":brewery["License Date"],
                "Status":brewery["Status"],
                "search_count":len(brewery["untappd"]),
                "brewery_name":entries["brewery"]["brewery_name"],
                "brewery_id":entries["brewery"]["brewery_id"],
                "beer_count":entries["brewery"]["beer_count"],
                "brewery_city":entries["brewery"]["location"]["brewery_city"],
                "brewery_state":entries["brewery"]["location"]["brewery_state"],
                "brewery_lat":entries["brewery"]["location"]["lat"],
                "brewery_lng":entries["brewery"]["location"]["lng"]
            })
    else:
        brewery_parsed.append({
            "Name":brewery["Name"],
            "Street Address":brewery["Street Address"],
            "Road":brewery["Road"],
            "City":brewery["City"],
            "State":brewery["State"],
            "Brewery License":brewery["Brewery License"],
            "License Date":brewery["License Date"],
            "Status":brewery["Status"],
            "search_count":0,
            "brewery_name":"",
            "brewery_id":"",
            "beer_count":0,
            "brewery_city":"",
            "brewery_state":"",
            "brewery_lat":float('NaN'),
            "brewery_lng":float('NaN')
        })
#         print(brewery)
#         print("##########")
#     break
        

# Remove results where not in texas
# brewery_parsed = [ brewery for brewery in brewery_parsed if "TX" in brewery["brewery_state"]]
# Must have 10 beers!
# brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["beer_count"] > 9 ]

#Convert to Pandas and save it
# brewery_parsed = pd.DataFrame( brewery_parsed )
# brewery_parsed.to_csv("../data/brewery_search.csv")
pd.options.display.max_rows = 999
display( pd.DataFrame( brewery_parsed ) )
pd.options.display.max_rows = 15

,Name,Street Address,Road,City,State,Brewery License,License Date,Status,search_count,brewery_name,brewery_id,beer_count,brewery_city,brewery_state,brewery_lat,brewery_lng
0,111 Brewing,111,S FANNIN AVENUE,DENISON,TX,Brewery,4/10/2019,In the works,1,Milepost 111 Brewing Company,66888,65,Cashmere,WA,47.5244,-120.470000
1,2 Docs Brewing Co,502,TEXAS AVENUE,LUBBOCK,TX,Brewery,10/5/2018,Open,1,Two Docs Brewing Co.,420745,22,Lubbock,TX,33.5909,-101.846000
2,3 Nations Brewing,2405,SQUIRE PLACE SUITE 200,FARMERS BRANCH,TX,Brewery,5/6/2015,Open,1,3 Nations Brewing,205645,43,Farmers Branch,TX,32.9310,-96.898300
3,4th Tap Brewing Cooperative,10615,METRIC BOULEVARD,AUSTIN,TX,Brewery,4/13/2015,Open,0,,,0,,,NaN,NaN
4,512 Brewing Company,407,RADAM LANE 'F200',AUSTIN,TX,Brewery,5/20/2008,Open,1,(512) Brewing Company,1,156,Austin,TX,30.2236,-97.769700
5,8th Wonder Brewery,2202,DALLAS STREET,HOUSTON,TX,Brewery,12/29/2011,Open,1,8th Wonder Brewery,45113,157,Houston,TX,29.7492,-95.355800
6,Alamo Beer,415,BURNET STREET,SAN ANTONIO,TX,Brewery,9/29/2014,Open,1,Alamo Beer Company,43137,69,San Antonio,TX,29.4295,-98.477300
7,Allen's Landing Brewing Company,3540,OAK FOREST DRIVE,HOUSTON,TX,Brewery,1/16/2015,Open,0,,,0,,,NaN,NaN
8,Altmeyer & Lewis Brewing Company,15898,HWY 123 NORTH,SAN MARCOS,TX,Brewery,7/16/2015,Open,1,Altmeyer & Lewis Brewing Company,294420,0,San Marcos,TX,29.7874,-97.946700
9,Armadillo Ale Works,221,S BELL AVENUE,DENTON,TX,Brewery,2/19/2015,Open,1,Armadillo Ale Works™,9316,28,Denton,TX,33.2126,-97.128100


4

In [3]:
breweryId = breweries[0]["brewery"]["brewery_id"]

untappd = UntappdApi(os.getenv("CLIENT_ID"),os.getenv("CLIENT_SECRET"))
brewery, counter, response = untappd.breweryInfo(breweryId)

brewery

{'brewery': {'brewery_id': 48372,
  'brewery_name': 'Community Beer Company',
  'brewery_slug': 'community-beer-company',
  'brewery_page_url': '/CommunityBeerCompanyTexas',
  'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
  'brewery_label_hd': '',
  'country_name': 'United States',
  'brewery_in_production': 0,
  'is_independent': 1,
  'claimed_status': {'is_claimed': True,
   'claimed_slug': 'CommunityBeerCompanyTexas',
   'follow_status': False,
   'follower_count': 10003,
   'uid': 514817,
   'mute_status': 'none'},
  'beer_count': 213,
  'contact': {'twitter': 'CommunityBeerCo',
   'facebook': 'http://www.facebook.com/communitybeer',
   'instagram': 'communitybeerco',
   'url': 'http://www.communitybeer.com'},
  'brewery_type': 'Regional Brewery',
  'brewery_type_id': 11,
  'location': {'brewery_address': '1530 Inspiration Dr',
   'brewery_city': 'Dallas',
   'brewery_state': 'TX',
   'brewery_lat': 32.7909,
   'brewery_lng': -96.8158

In [59]:
entries

[{'brewery': {'brewery_id': 222679,
   'beer_count': 76,
   'brewery_name': 'Westlake Brewing Company',
   'brewery_slug': 'westlake-brewing-company',
   'brewery_page_url': '/WestlakeBrewingCompany',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-222679_50302.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Westlake Village',
    'brewery_state': 'CA',
    'lat': 34.1541,
    'lng': -118.802}}},
 {'brewery': {'brewery_id': 305108,
   'beer_count': 29,
   'brewery_name': 'Westlake Brewing Company',
   'brewery_slug': 'westlake-brewing-company',
   'brewery_page_url': '/Westlake_Brewing_Company',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-305108_24443.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Dallas',
    'brewery_state': 'TX',
    'lat': 32.7831,
    'lng': -96.7824}}},
 {'brewery': {'brewery_id': 196104,
   'beer_count': 10,
   'brewery_name': 'West Lake Brewi

In [19]:
from dotenv import load_dotenv
import os

untappd = Untappd()

breweries, counter, response = untappd.findBrewery("Community Beer Company")

breweries

[{'brewery': {'brewery_id': 48372,
   'beer_count': 213,
   'brewery_name': 'Community Beer Company',
   'brewery_slug': 'community-beer-company',
   'brewery_page_url': '/CommunityBeerCompanyTexas',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Dallas',
    'brewery_state': 'TX',
    'lat': 32.7909,
    'lng': -96.8158}}},
 {'brewery': {'brewery_id': 259927,
   'beer_count': 1,
   'brewery_name': 'The Community Beer Company',
   'brewery_slug': 'the-community-beer-company',
   'brewery_page_url': '/w/the-community-beer-company/259927',
   'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png',
   'country_name': 'United States',
   'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}]

In [63]:
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

i = 0
for brewery in all_breweries:
    if "untappd" in brewery:
        clear_output()
        i+=1
        print(i)

# with open("../data/all_breweries.pkl", "wb") as fp:
#     pickle.dump(all_breweries, fp)

347


In [54]:
from uszipcode import SearchEngine

for brewery in brewery_parsed:
    if abs(brewery["brewery_lat"]) > 0 and abs(brewery["brewery_lng"]) > 0:
        search = SearchEngine(simple_zipcode=False)
        result = search.by_coordinates(brewery["brewery_lat"],brewery["brewery_lng"], radius=30, returns=1)
        result = result[0].to_dict()
        display( {
            "brewery_city":brewery["brewery_city"],
            "major_city":result["major_city"],
            "state":result["state"],
            "zipcode":result["zipcode"]
        } )
    
    

{'brewery_city': 'Cashmere',
 'major_city': 'Cashmere',
 'state': 'WA',
 'zipcode': '98815'}

{'brewery_city': 'Lubbock',
 'major_city': 'Lubbock',
 'state': 'TX',
 'zipcode': '79401'}

{'brewery_city': 'Farmers Branch',
 'major_city': 'Dallas',
 'state': 'TX',
 'zipcode': '75234'}

{'brewery_city': 'Austin',
 'major_city': 'Austin',
 'state': 'TX',
 'zipcode': '78704'}

{'brewery_city': 'Houston',
 'major_city': 'Houston',
 'state': 'TX',
 'zipcode': '77003'}

...

{'brewery_city': 'Katy',
 'major_city': 'Katy',
 'state': 'TX',
 'zipcode': '77494'}

{'brewery_city': 'Tomball',
 'major_city': 'Houston',
 'state': 'TX',
 'zipcode': '77070'}

{'brewery_city': 'Galveston',
 'major_city': 'Galveston',
 'state': 'TX',
 'zipcode': '77550'}

{'brewery_city': 'Pharr',
 'major_city': 'Pharr',
 'state': 'TX',
 'zipcode': '78577'}

IndexError: list index out of range

In [60]:
all_breweries

[{'Name': '111 Brewing',
  'Street Address': 111,
  'Road': 'S FANNIN AVENUE',
  'City': 'DENISON',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '4/10/2019',
  'Status': 'In the works',
  'untappd': [{'brewery': {'brewery_id': 66888,
     'beer_count': 65,
     'brewery_name': 'Milepost 111 Brewing Company',
     'brewery_slug': 'milepost-111-brewing-company',
     'brewery_page_url': '/milepostbeer',
     'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg',
     'country_name': 'United States',
     'location': {'brewery_city': 'Cashmere',
      'brewery_state': 'WA',
      'lat': 47.5244,
      'lng': -120.47}}}]},
 {'Name': '2 Docs Brewing Co',
  'Street Address': 502,
  'Road': 'TEXAS AVENUE',
  'City': 'LUBBOCK',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '10/5/2018',
  'Status': 'Open',
  'untappd': [{'brewery': {'brewery_id': 420745,
     'beer_count': 22,
     'brewery_name': 'Two